In [1]:
# 使用bagging

In [11]:
import tensorflow as tf
import numpy as np
import sklearn.feature_selection as fs

TRAIN_DATA_FILE = "train_data.csv"
TEST_DATA_FILE = "test_data.csv"


def read_split_train_test_data():
    training_set = tf.contrib.learn.datasets.base.load_csv_with_header(
        filename=TRAIN_DATA_FILE,
        target_dtype=np.int,
        features_dtype=np.float32,
        target_column=0
    )
    test_set = tf.contrib.learn.datasets.base.load_csv_with_header(
        filename=TEST_DATA_FILE,
        target_dtype=np.int,
        features_dtype=np.float32,
        target_column=0
    )
    # print(training_set)
    X, y = training_set.data, training_set.target
    print(X.shape)
    print(y.shape)
    X_test, y_test = test_set.data, test_set.target
    print(X_test.shape)
    print(y_test.shape)
    return X, y, X_test, y_test


def read_train_test_data():
    '''不把标签和数据分开'''
    x, y, x_t, y_t = read_split_train_test_data()
    y = y.reshape(x.shape[0], -1)
    print(y.shape)
    y_t = y_t.reshape(x_t.shape[0], -1)
    x_train = np.concatenate((x, y), axis=1)
    x_test = np.concatenate((x_t, y_t), axis=1)
    print(x_train.shape)
    return x_train, x_test


def split_data_label(X):
    '''将数据和标签分开,默认标签位于最后一列'''
    data, label = X[:, :-1], X[:, -1]
    return data, label


# print(training_set)
X, y, X_test, y_test = read_split_train_test_data()
print(X.shape)
print(y_test.shape)
d_train, d_test = read_train_test_data()
print(d_train.shape)


(10545, 28)
(10545,)
(300, 28)
(300,)
(10545, 28)
(300,)
(10545, 28)
(10545,)
(300, 28)
(300,)
(10545, 1)
(10545, 29)
(10545, 29)


In [3]:
from sklearn.naive_bayes import GaussianNB
from sklearn import metrics
from sklearn.ensemble import BaggingClassifier
from collections import Counter

In [4]:
cls = BaggingClassifier(base_estimator=GaussianNB(),max_samples=1.0)

y_pred = cls.fit(X, y).predict(X_test)
acc = metrics.accuracy_score(y_test, y_pred)
print(acc)


0.663333333333


In [5]:
# 打乱数据集

cls = BaggingClassifier(base_estimator=GaussianNB(), max_samples=1.0)

indices = np.random.permutation(X.shape[0])  # shape[0]表示第0轴的长度，通常是训练数据的数量  
rand_data_x = X[indices]
rand_data_y = y[indices]  # data_y就是标记（label）  

y_pred = cls.fit(rand_data_x, rand_data_y).predict(X_test)
print(type(y_pred))
acc = metrics.accuracy_score(y_test, y_pred)
print(acc)


<class 'numpy.ndarray'>
0.676666666667


In [6]:
# 我打算将数据手动分开再分别训练

In [7]:
def split_class_1(X, y):
    # 提取出类别为1的数据
    X_1 = np.array([list(X[i]) for i in range(len(y)) if y[i] == 1])
    y_1 = np.array([y[i] for i in range(len(y)) if y[i] == 1])
    print(X_1.shape[0], y_1.shape[0])
    # 提取出类别不为1的数据
    X_n1 = np.array([list(X[i]) for i in range(len(y)) if y[i] != 1])
    y_n1 = np.array([y[i] for i in range(len(y)) if y[i] != 1])
    return X_1, y_1, X_n1, y_n1
# print(len(X_n1), len(y_n1))

7431 7431
3114 3114


In [8]:
pred_record = []
y_pred = None
al = int(len(X_1) / 7)
gnb = GaussianNB()
for i in range(6):
    xt1 = X_1[i * al:(i + 1) * al, :]
    yt1 = y_1[i * al:(i + 1) * al]
    X_temp = np.concatenate((xt1, X_n1), axis=0)
    y_temp = np.concatenate((yt1, y_n1), axis=0)
    # print(temp.shape)
    if i == 0:
        y_pred = gnb.fit(X_temp, y_temp).predict(X_test)
    else:
        y_p = gnb.fit(X_temp, y_temp).predict(X_test)
        y_pred = np.concatenate((y_pred, y_p))

        # acc = metrics.accuracy_score(y_test, y_pred)
        # print(acc)
y_pred = y_pred.reshape(6, -1)
print(y_pred.shape)

col_len = y_pred.shape[1]
# col_len = 5
y_pred_final = []
for i in range(col_len):
    choose = Counter(list(y_pred[:, i])).most_common(1)[0][0]
    # print(choose)
    y_pred_final.append(choose)

acc = metrics.accuracy_score(y_test, y_pred_final)
print(acc)


(6, 300)
0.693333333333


In [9]:

al = int(len(X_1) / 4)
gnb = GaussianNB()
for i in range(4):
    xt1 = X_1[i * al:(i + 1) * al, :]
    yt1 = y_1[i * al:(i + 1) * al]
    X_temp = np.concatenate((xt1, X_n1), axis=0)
    y_temp = np.concatenate((yt1, y_n1), axis=0)
    # print(temp.shape)
    y_pred = gnb.fit(X_temp, y_temp).predict(X_test)
    
    # 记录每个结果,最后会评比
    
    acc = metrics.accuracy_score(y_test, y_pred)
    print(acc)


0.71
0.7
0.62
0.586666666667
